In [ ]:
# Spring 2022, IOC 5259 Reinforcement Learning
# HW1-partII: REINFORCE and baseline

import gym
from itertools import count
from collections import namedtuple
import numpy as np
from numpy import sqrt 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import torch.optim.lr_scheduler as Scheduler


# Define a useful tuple (optional)
SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])

        
class Policy(nn.Module):
    """
        Implement both policy network and the value network in one model
        - Note that here we let the actor and value networks share the first layer
        - Feel free to change the architecture (e.g. number of hidden layers and the width of each hidden layer) as you like
        - Feel free to add any member variables/functions whenever needed
        TODO:
            1. Initialize the network (including the shared layer(s), the action layer(s), and the value layer(s)
            2. Random weight initialization of each layer
    """
    def __init__(self):
        super(Policy, self).__init__()
        
        # Extract the dimensionality of state and action spaces
        self.discrete = isinstance(env.action_space, gym.spaces.Discrete)
        self.observation_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.n if self.discrete else env.action_space.shape[0]
        self.hidden_size = 128
#         print(self.observation_dim )
#         print(self.action_dim )
        ########## YOUR CODE HERE (5~10 lines) ##########
        self.s_layer1= nn.Linear(8, self.hidden_size)
        torch.nn.init.xavier_uniform_(self.s_layer1.weight)
        
        self.s_layer2= nn.Linear(self.hidden_size, self.hidden_size)
        torch.nn.init.xavier_uniform_(self.s_layer2.weight)

        self.a_layer = nn.Linear(self.hidden_size, 4)
        torch.nn.init.xavier_uniform_(self.a_layer.weight)

        
        self.v_layer = nn.Linear(self.hidden_size, 1)
        torch.nn.init.xavier_uniform_(self.v_layer.weight)

        ########## END OF YOUR CODE ##########
        
        # action & reward memory
        self.saved_actions = []
        self.rewards = []

    def forward(self, state):
        """
            Forward pass of both policy and value networks
            - The input is the state, and the outputs are the corresponding 
              action probability distirbution and the state value
            TODO:
                1. Implement the forward pass for both the action and the state value
        """
        
        ########## YOUR CODE HERE (3~5 lines) ##########
        x =  self.s_layer1(state)
        x = F.relu(x)
        x =  self.s_layer2(x)
        x = F.relu(x)
        y=self.a_layer(x)
        action_prob = F.softmax(y, dim=-1)#choses what action you want to take and return probability of each action
        state_value = self.v_layer(x)
         
        # return values are  tuple, action_prob:  the probability of each action over the action space
                                          #  state_value: the value from state s_t 
        ########## END OF YOUR CODE ##########

        return action_prob, state_value




    def select_action(self, state):
        """
            Select the action given the current state
            - The input is the state, and the output is the action to apply 
            (based on the learned stochastic policy)
            TODO:
                1. Implement the forward pass for both the action and the state value
        """
        
        ########## YOUR CODE HERE (3~5 lines) ##########
        state = torch.from_numpy(state).float().squeeze(0)
        action_prob, state_value = self.forward(state)
        m = Categorical(action_prob)       #create a distribution from action_prob
        action = m.sample()    #sample the action

        ########## END OF YOUR CODE ##########
        
        # save to action buffer
        self.saved_actions.append(SavedAction(m.log_prob(action), state_value))

        return action.item()


    def calculate_loss(self, gamma=0.99):
        """
            Calculate the loss (= policy loss + value loss) to perform backprop later
            TODO:
                1. Calculate rewards-to-go required by REINFORCE with the help of self.rewards
                2. Calculate the policy loss using the policy gradient
                3. Calculate the value loss using either MSE loss or smooth L1 loss
        """
        
        # Initialize the lists and variables
        R = 0
        saved_actions = self.saved_actions
        policy_losses = [] 
        value_losses = [] 
        returns = []

        ########## YOUR CODE HERE (8-15 lines) ##########
        eps = np.finfo(np.float32).eps.item()

        for r in reversed(self.rewards):
            R = r + gamma * R
            returns.insert(0, R)        #calculate discounted value
        
        returns = torch.tensor(returns).float()
        returns = (returns - returns.mean()) / (returns.std() + eps) #standardlization for faster converge / eps: To increase the stability when calculating
        
        
        for (log_prob, state_value),R in zip(saved_actions , returns):
            advantage = R - state_value.item()                        
            policy_losses.append(advantage * -log_prob )        #using baseline to reduce variance (baseline=v_pi(s)) with advantage function
            
            value_losses.append( F.mse_loss(state_value, torch.tensor ([R]) ) )    # calcilate loss using MSE
            
        loss = torch.stack(policy_losses).sum()+torch.stack(value_losses).sum()    #sum up both policy_losses and value_losses
 
        ########## END OF YOUR CODE ##########
        
        return loss

    def clear_memory(self):
        # reset rewards and action buffer
        del self.rewards[:]
        del self.saved_actions[:]


def train(lr=0.005):
    '''
        Train the model using SGD (via backpropagation)
        TODO: In each episode, 
        1. run the policy till the end of the episode and keep the sampled trajectory
        2. update both the policy and the value network at the end of episode
    '''    
    
    # Instantiate the policy model and the optimizer
    model = Policy()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Learning rate scheduler (optional)
#     scheduler = Scheduler.StepLR(optimizer, step_size=100, gamma=0.9)
    
    # EWMA reward for tracking the learning progress
    ewma_reward = 0
    
    # run inifinitely many episodes
    for i_episode in count(1):
        # reset environment and episode reward
        state = env.reset()
        ep_reward = 0
        t = 0
        # Uncomment the following line to use learning rate scheduler
        
        
        # For each episode, only run 9999 steps so that we don't 
        # infinite loop while learning
        
        ########## YOUR CODE HERE (10-15 lines) ##########
        for t in range(0, 10000):  # Don't infinite loop while learning
            action = model.select_action(state)
            s_state, reward, done, _ = env.step(action)

            model.rewards.append(reward)
            ep_reward += reward
            if done:
                break
            state = s_state

        # Backpropagation
        optimizer.zero_grad()
        policy_loss = model.calculate_loss()
        policy_loss.backward()
        optimizer.step()
#         scheduler.step()
        model.clear_memory()
        
        ########## END OF YOUR CODE ##########
            
        # update EWMA reward and log the results
        ewma_reward = 0.05 * ep_reward + (1 - 0.05) * ewma_reward
        print('Episode {}\tlength: {}\treward: {}\t ewma reward: {}'.format(i_episode, t, ep_reward, ewma_reward))

        # check if we have "solved" the cart pole problem
        if ewma_reward > env.spec.reward_threshold:
            torch.save(model.state_dict(), './preTrained/LunarLander_{}.pth'.format(lr))
#             torch.save(model.state_dict(), './preTrained/LunarLander.pth')
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(ewma_reward, t))
            break


def test(name, n_episodes=10):
    '''
        Test the learned model (no change needed)
    '''      
    model = Policy()
    
    model.load_state_dict(torch.load('./preTrained/{}'.format(name)))
    
    render = True
    max_episode_len = 10000
    
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        running_reward = 0
        for t in range(max_episode_len+1):
            action = model.select_action(state)
            state, reward, done, _ = env.step(action)
            running_reward += reward
            if render:
                 env.render()
            if done:
                break
        print('Episode {}\tReward: {}'.format(i_episode, running_reward))
    env.close()
    

if __name__ == '__main__':
    # For reproducibility, fix the random seed
    random_seed = 20  
    lr = 0.005
    env = gym.make("LunarLander-v2")
    env.seed(random_seed)  
    torch.manual_seed(random_seed)  
    train(lr)
#     test('LunarLander_0.01.pth')
    test('LunarLander_0.005.pth')


Episode 1	length: 72	reward: -72.10817345845335	 ewma reward: -3.6054086729226675
Episode 2	length: 94	reward: -113.68456511415435	 ewma reward: -9.109366494984252
Episode 3	length: 83	reward: -102.141332726004	 ewma reward: -13.76096480653524
Episode 4	length: 111	reward: -346.97170635128003	 ewma reward: -30.42150188377248
Episode 5	length: 82	reward: -146.29018118443864	 ewma reward: -36.21493584880579
Episode 6	length: 78	reward: -162.42387272482648	 ewma reward: -42.52538269260682
Episode 7	length: 58	reward: -139.97343629718094	 ewma reward: -47.39778537283553
Episode 8	length: 109	reward: -223.32553462623972	 ewma reward: -56.19417283550574
Episode 9	length: 89	reward: -383.6158308745566	 ewma reward: -72.56525573745829
Episode 10	length: 83	reward: -112.36755045253813	 ewma reward: -74.55537047321226
Episode 11	length: 120	reward: -156.0931668717655	 ewma reward: -78.63226029313992
Episode 12	length: 116	reward: -472.44252443231045	 ewma reward: -98.32277350009844
Episode 13	le

Episode 100	length: 115	reward: -109.5949136451231	 ewma reward: -168.72447137979285
Episode 101	length: 155	reward: -36.149710983267966	 ewma reward: -162.0957333599666
Episode 102	length: 88	reward: -77.86623811687208	 ewma reward: -157.88425859781188
Episode 103	length: 95	reward: -197.96785317441126	 ewma reward: -159.88843832664182
Episode 104	length: 99	reward: -115.29581357784807	 ewma reward: -157.65880708920213
Episode 105	length: 76	reward: -101.82924765486446	 ewma reward: -154.86732911748524
Episode 106	length: 141	reward: -117.16883588523575	 ewma reward: -152.98240445587274
Episode 107	length: 136	reward: -106.34869904244873	 ewma reward: -150.65071918520152
Episode 108	length: 126	reward: -158.01813396783655	 ewma reward: -151.01908992433326
Episode 109	length: 123	reward: -232.98194636585256	 ewma reward: -155.11723274640923
Episode 110	length: 108	reward: -101.41545415497633	 ewma reward: -152.4321438168376
Episode 111	length: 117	reward: -165.66960217022313	 ewma rewa

Episode 198	length: 93	reward: -187.16821760313724	 ewma reward: -134.01752276639803
Episode 199	length: 101	reward: 3.758925772215079	 ewma reward: -127.12870033946736
Episode 200	length: 79	reward: -87.31566821154193	 ewma reward: -125.13804873307107
Episode 201	length: 110	reward: -267.51301890097545	 ewma reward: -132.25679724146627
Episode 202	length: 72	reward: -93.17825231427523	 ewma reward: -130.30286999510673
Episode 203	length: 67	reward: -109.13155888747808	 ewma reward: -129.24430443972528
Episode 204	length: 66	reward: -86.5372587921235	 ewma reward: -127.10895215734519
Episode 205	length: 145	reward: -78.89657212600002	 ewma reward: -124.69833315577793
Episode 206	length: 124	reward: -282.23318109131077	 ewma reward: -132.57507555255458
Episode 207	length: 117	reward: -201.69381313271288	 ewma reward: -136.0310124315625
Episode 208	length: 148	reward: -88.0731966508755	 ewma reward: -133.63312164252812
Episode 209	length: 122	reward: -192.4712361470032	 ewma reward: -136

Episode 295	length: 217	reward: -128.88466875998085	 ewma reward: -122.76419444577672
Episode 296	length: 143	reward: -149.74252661844497	 ewma reward: -124.11311105441014
Episode 297	length: 165	reward: -152.8057669389351	 ewma reward: -125.54774384863639
Episode 298	length: 108	reward: -100.72889119744788	 ewma reward: -124.30680121607695
Episode 299	length: 158	reward: -69.69174658594436	 ewma reward: -121.57604848457031
Episode 300	length: 194	reward: -153.77995001089218	 ewma reward: -123.1862435608864
Episode 301	length: 89	reward: -61.57606320461984	 ewma reward: -120.10573454307305
Episode 302	length: 221	reward: -168.30134074434147	 ewma reward: -122.51551485313647
Episode 303	length: 184	reward: -156.2175308207734	 ewma reward: -124.2006156515183
Episode 304	length: 108	reward: -67.03119781858891	 ewma reward: -121.34214475987183
Episode 305	length: 124	reward: -79.18392965478102	 ewma reward: -119.23423400461728
Episode 306	length: 107	reward: -100.39558765811	 ewma reward: 

Episode 392	length: 161	reward: -33.80583044116456	 ewma reward: -97.01602175535838
Episode 393	length: 112	reward: -134.08618981869077	 ewma reward: -98.869530158525
Episode 394	length: 100	reward: -71.13155723720166	 ewma reward: -97.48263151245882
Episode 395	length: 175	reward: -56.735042988251806	 ewma reward: -95.44525208624847
Episode 396	length: 146	reward: -50.40664450149496	 ewma reward: -93.1933217070108
Episode 397	length: 124	reward: -116.27679701445004	 ewma reward: -94.34749547238276
Episode 398	length: 104	reward: 4.669153952293925	 ewma reward: -89.39666300114892
Episode 399	length: 126	reward: -41.268755835383665	 ewma reward: -86.99026764286066
Episode 400	length: 88	reward: -39.37944581541733	 ewma reward: -84.60972655148849
Episode 401	length: 100	reward: -51.41978906534192	 ewma reward: -82.95022967718117
Episode 402	length: 142	reward: -53.02032638204388	 ewma reward: -81.4537345124243
Episode 403	length: 590	reward: -391.3657902835217	 ewma reward: -96.949337300

...

Episode 4795	length: 999	reward: 147.34869134864599	 ewma reward: 159.2152688919361
Episode 4796	length: 718	reward: 267.81156891117547	 ewma reward: 164.64508389289807
Episode 4797	length: 999	reward: 173.1001058924575	 ewma reward: 165.06783499287602
Episode 4798	length: 999	reward: 132.1091205290784	 ewma reward: 163.4198992696861
Episode 4799	length: 999	reward: 152.42489439834608	 ewma reward: 162.87014902611912
Episode 4800	length: 999	reward: 119.04112980713637	 ewma reward: 160.67869806516998
Episode 4801	length: 999	reward: 124.4031505606169	 ewma reward: 158.86492068994232
Episode 4802	length: 999	reward: 121.76760618533203	 ewma reward: 157.0100549647118
Episode 4803	length: 999	reward: 171.4130978341351	 ewma reward: 157.73020710818298
Episode 4804	length: 999	reward: 140.94650869259374	 ewma reward: 156.89102218740354
Episode 4805	length: 147	reward: 37.112035048990634	 ewma reward: 150.9020728304829
Episode 4806	length: 999	reward: 161.35995011714672	 ewma reward: 151.424

Episode 4892	length: 999	reward: 152.361401906033	 ewma reward: 159.79243990257055
Episode 4893	length: 999	reward: 187.1788721042945	 ewma reward: 161.16176151265674
Episode 4894	length: 999	reward: 144.78966017021142	 ewma reward: 160.34315644553448
Episode 4895	length: 999	reward: 169.93422370920504	 ewma reward: 160.82270980871797
Episode 4896	length: 999	reward: 173.34911196206974	 ewma reward: 161.44902991638554
Episode 4897	length: 999	reward: 110.5127156516077	 ewma reward: 158.90221420314666
Episode 4898	length: 999	reward: 146.043948977	 ewma reward: 158.2593009418393
Episode 4899	length: 999	reward: 155.3054430256063	 ewma reward: 158.11160804602767
Episode 4900	length: 999	reward: 147.28659604252059	 ewma reward: 157.57035744585232
Episode 4901	length: 999	reward: 147.7768725085451	 ewma reward: 157.08068319898695
Episode 4902	length: 999	reward: 111.99261389897508	 ewma reward: 154.82627973398635
Episode 4903	length: 999	reward: 163.44733041975672	 ewma reward: 155.2573322

Episode 4990	length: 999	reward: 185.0218292391218	 ewma reward: 159.18644426885277
Episode 4991	length: 327	reward: 286.83180883190187	 ewma reward: 165.5687124970052
Episode 4992	length: 999	reward: 166.82548534607545	 ewma reward: 165.6315511394587
Episode 4993	length: 999	reward: 150.8052560391569	 ewma reward: 164.89023638444363
Episode 4994	length: 999	reward: 129.85082980677802	 ewma reward: 163.13826605556034
Episode 4995	length: 999	reward: 178.7190777591905	 ewma reward: 163.91730664074183
Episode 4996	length: 999	reward: 183.72864412431738	 ewma reward: 164.9078735149206
Episode 4997	length: 999	reward: 160.39784275235007	 ewma reward: 164.68237197679207
Episode 4998	length: 999	reward: 166.48211077690803	 ewma reward: 164.77235891679786
Episode 4999	length: 999	reward: 171.17406116862733	 ewma reward: 165.09244402938933
Episode 5000	length: 999	reward: 176.87688716938973	 ewma reward: 165.68166618638932
Episode 5001	length: 999	reward: 193.76571421045236	 ewma reward: 167.0

Episode 5088	length: 999	reward: 146.14656370169257	 ewma reward: 150.60395259232328
Episode 5089	length: 999	reward: 186.82284589786588	 ewma reward: 152.41489725760042
Episode 5090	length: 999	reward: 161.30262956342682	 ewma reward: 152.85928387289172
Episode 5091	length: 999	reward: 180.82870375527926	 ewma reward: 154.2577548670111
Episode 5092	length: 999	reward: 122.48514231618176	 ewma reward: 152.66912423946962
Episode 5093	length: 999	reward: 122.36429407419453	 ewma reward: 151.15388273120587
Episode 5094	length: 999	reward: 169.12408967567262	 ewma reward: 152.05239307842922
Episode 5095	length: 999	reward: 179.86728686765102	 ewma reward: 153.4431377678903
Episode 5096	length: 999	reward: 126.12034299061186	 ewma reward: 152.07699802902638
Episode 5097	length: 999	reward: 152.99686262722517	 ewma reward: 152.1229912589363
Episode 5098	length: 999	reward: 173.2148125205768	 ewma reward: 153.17758232201834
Episode 5099	length: 999	reward: 130.2716669340315	 ewma reward: 152.

Episode 5185	length: 999	reward: 161.42416361769276	 ewma reward: 154.2939168826299
Episode 5186	length: 999	reward: 172.43662201438838	 ewma reward: 155.2010521392178
Episode 5187	length: 999	reward: 146.72081879812959	 ewma reward: 154.77704047216338
Episode 5188	length: 999	reward: 132.5970279583732	 ewma reward: 153.66803984647385
Episode 5189	length: 999	reward: 147.84881189927626	 ewma reward: 153.37707844911395
Episode 5190	length: 999	reward: 199.25357707894543	 ewma reward: 155.67090338060552
Episode 5191	length: 999	reward: 171.43908038323073	 ewma reward: 156.45931223073677
Episode 5192	length: 999	reward: 143.90411794587615	 ewma reward: 155.83155251649373
Episode 5193	length: 999	reward: 114.0517706761302	 ewma reward: 153.74256342447555
Episode 5194	length: 999	reward: 171.54183659370287	 ewma reward: 154.63252708293692
Episode 5195	length: 999	reward: 164.14792251483118	 ewma reward: 155.10829685453163
Episode 5196	length: 999	reward: 122.14373594275769	 ewma reward: 153